# Setup

In [1]:
%load_ext autoreload
import pandas as pd
import numpy as np
from math import radians, sin, cos, asin, sqrt, pi

In [2]:
!pip install numba

You should consider upgrading via the '/Users/olivergiles/.pyenv/versions/3.8.12/envs/lewagon/bin/python3.8 -m pip install --upgrade pip' command.


In [3]:
url = 's3://wagon-public-datasets/taxi-fare-train.csv'
df = pd.read_csv(url, nrows=500_000)
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [4]:
assert(df.shape[0] == 500_000)

Haversine function from olist

In [5]:
def haversine(lon1,lat1,lon2,lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    return 2 * 6371 * asin(sqrt(a))


## Numba 

### Baseline

In [6]:
%%time
df.apply(lambda x: haversine(x["pickup_longitude"], x["pickup_latitude"], 
            x["dropoff_longitude"], x["dropoff_latitude"]), axis=1)

CPU times: user 15.2 s, sys: 462 ms, total: 15.7 s
Wall time: 19.5 s


0         1.030764
1         8.450134
2         1.389525
3         2.799270
4         1.999157
            ...   
499995    1.404709
499996    0.994184
499997    7.859324
499998    0.994470
499999    1.040272
Length: 500000, dtype: float64

In [7]:
from numba import jit

@jit
def jit_haversine(lon1,lat1,lon2,lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    return 2 * 6371 * asin(sqrt(a))

In [8]:
%%timeit -r 3 
haversine(1,2,3,4)

1.95 µs ± 85.7 ns per loop (mean ± std. dev. of 3 runs, 1000000 loops each)


In [10]:
%%timeit -r 3 
jit_haversine(1,2,3,4)

812 ns ± 51.7 ns per loop (mean ± std. dev. of 3 runs, 1000000 loops each)


In [11]:
%%time
df.apply(lambda x: jit_haversine(x["pickup_longitude"], x["pickup_latitude"], 
            x["dropoff_longitude"], x["dropoff_latitude"]), axis=1)

CPU times: user 15.2 s, sys: 498 ms, total: 15.7 s
Wall time: 18.8 s


0         1.030764
1         8.450134
2         1.389525
3         2.799270
4         1.999157
            ...   
499995    1.404709
499996    0.994184
499997    7.859324
499998    0.994470
499999    1.040272
Length: 500000, dtype: float64

## Row major v Column major

In [12]:
small_df = df.head(10_000)

In [13]:
%%time
# iterate pandas DataFrame by columns
for col in small_df.columns:
    for el in small_df[col]:
        pass

CPU times: user 20.5 ms, sys: 764 µs, total: 21.3 ms
Wall time: 25 ms


In [14]:
%%time
# iterate pandas DataFrame by rows
for r in range(small_df.shape[0]):
    for el in small_df.iloc[r]:
        pass

CPU times: user 1.51 s, sys: 28.1 ms, total: 1.54 s
Wall time: 1.77 s


<img src="row_column_wise.png" width=700>

## More numba

In [15]:
%%time
hav = np.zeros(500_000)
np_data = np.array(df[["pickup_longitude", "pickup_latitude", "dropoff_longitude","dropoff_latitude"]])
for i, sample in enumerate(np_data):
    hav[i] = jit_haversine(sample[0],sample[1],sample[2],sample[3])
hav

CPU times: user 1.11 s, sys: 56.3 ms, total: 1.17 s
Wall time: 1.4 s


array([1.03076394, 8.4501336 , 1.38952523, ..., 7.85932354, 0.99447026,
       1.04027174])

In [16]:
%%time
distances = np.array(df[["pickup_longitude", "pickup_latitude", "dropoff_longitude","dropoff_latitude"]])
@jit
def haversine_loop(data):
    hav = np.zeros(len(data))
    for i, sample in enumerate(data):
        hav[i] = jit_haversine(sample[0],sample[1],sample[2],sample[3])
    return hav
haversine_loop(distances)

CPU times: user 559 ms, sys: 34.9 ms, total: 594 ms
Wall time: 720 ms


array([1.03076394, 8.4501336 , 1.38952523, ..., 7.85932354, 0.99447026,
       1.04027174])

In [18]:
from numba import vectorize, float64

In [19]:
@vectorize
def vectorize_haversine(lon1,lat1,lon2,lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    return 2 * 6371 * asin(sqrt(a))

In [20]:
%%time
vectorize_haversine(df["pickup_longitude"], df["pickup_latitude"], 
            df["dropoff_longitude"], df["dropoff_latitude"])

ValueError: [1mCannot determine Numba type of <class 'pandas.core.series.Series'>[0m

In [21]:
@vectorize([float64(float64, float64, float64, float64)],cache=True)
def f64_haversine(lon1,lat1,lon2,lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    return 2 * 6371 * asin(sqrt(a))

In [22]:
%%time
f64_haversine(df["pickup_longitude"], df["pickup_latitude"], 
            df["dropoff_longitude"], df["dropoff_latitude"])

CPU times: user 168 ms, sys: 27.8 ms, total: 196 ms
Wall time: 261 ms


0         1.030764
1         8.450134
2         1.389525
3         2.799270
4         1.999157
            ...   
499995    1.404709
499996    0.994184
499997    7.859324
499998    0.994470
499999    1.040272
Length: 500000, dtype: float64

In [23]:
@vectorize([float64(float64, float64, float64, float64)],target="parallel")
def parallel_haversine(lon1,lat1,lon2,lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    return 2 * 6371 * asin(sqrt(a))

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [24]:
%%time
parallel_haversine(df["pickup_longitude"], df["pickup_latitude"], 
            df["dropoff_longitude"], df["dropoff_latitude"])

CPU times: user 178 ms, sys: 30.2 ms, total: 208 ms
Wall time: 266 ms


0         1.030764
1         8.450134
2         1.389525
3         2.799270
4         1.999157
            ...   
499995    1.404709
499996    0.994184
499997    7.859324
499998    0.994470
499999    1.040272
Length: 500000, dtype: float64

In [25]:
def distances_vectorized(start_lat, start_lon, end_lat, end_lon):
        """
        Calculate the haverzine and manhattan distance between two points on the earth (specified in decimal degrees).
        Vectorized version for pandas df
        Computes distance in kms
        """
        earth_radius = 6371

        lat_1_rad, lon_1_rad = np.radians(start_lat), np.radians(start_lon)
        lat_2_rad, lon_2_rad = np.radians(end_lat), np.radians(end_lon)

        dlon_rad = lon_2_rad - lon_1_rad
        dlat_rad = lat_2_rad - lat_1_rad
        
        a = (np.sin(dlat_rad / 2.0)**2 + np.cos(lat_1_rad) * np.cos(lat_2_rad) * np.sin(dlon_rad / 2.0)**2)
        haversine_rad = 2 * np.arcsin(np.sqrt(a))
        haversine_km = haversine_rad * earth_radius

        return haversine_km

In [26]:
%%time
distances_vectorized(df["pickup_longitude"], df["pickup_latitude"], 
            df["dropoff_longitude"], df["dropoff_latitude"])

CPU times: user 67.6 ms, sys: 17.4 ms, total: 85 ms
Wall time: 123 ms


0         0.410442
1         4.628504
2         1.001022
3         0.910440
4         1.361021
            ...   
499995    1.434688
499996    0.759068
499997    9.748259
499998    0.554714
499999    0.518838
Length: 500000, dtype: float64

In [27]:
@vectorize([float64(float64, float64, float64, float64)])
def numba_distances_vectorized(start_lat, start_lon, end_lat, end_lon):
        """
        Calculate the haverzine and manhattan distance between two points on the earth (specified in decimal degrees).
        Vectorized version for pandas df
        Computes distance in kms
        """
        earth_radius = 6371

        lat_1_rad, lon_1_rad = np.radians(start_lat), np.radians(start_lon)
        lat_2_rad, lon_2_rad = np.radians(end_lat), np.radians(end_lon)

        dlon_rad = lon_2_rad - lon_1_rad
        dlat_rad = lat_2_rad - lat_1_rad
        
        a = (np.sin(dlat_rad / 2.0)**2 + np.cos(lat_1_rad) * np.cos(lat_2_rad) * np.sin(dlon_rad / 2.0)**2)
        haversine_rad = 2 * np.arcsin(np.sqrt(a))
        haversine_km = haversine_rad * earth_radius

        return haversine_km

In [28]:
%%time
numba_distances_vectorized(df["pickup_longitude"], df["pickup_latitude"], 
            df["dropoff_longitude"], df["dropoff_latitude"])

CPU times: user 72.1 ms, sys: 5.16 ms, total: 77.3 ms
Wall time: 101 ms


0         0.410442
1         4.628504
2         1.001022
3         0.910440
4         1.361021
            ...   
499995    1.434688
499996    0.759068
499997    9.748259
499998    0.554714
499999    0.518838
Length: 500000, dtype: float64

# Cython

In [29]:
%load_ext Cython

In [32]:
%%cython -a

from libc.math cimport sin, cos, asin, sqrt
   
## Equivalent to 3.1415927 / 180
cdef float PI_RATIO = 0.017453293

cdef float deg2rad(float deg):
    cdef float rad = deg * PI_RATIO
    return rad
    
def cython_haversine(float lon1, float lat1, float lon2, float lat2):
    cdef float rlon1 = deg2rad(lon1)
    cdef float rlon2 = deg2rad(lon2)
    cdef float rlat1 = deg2rad(lat1)
    cdef float rlat2 = deg2rad(lat2)
    
    cdef float dlon = rlon2 - rlon1
    cdef float dlat = rlat2 - rlat1

    cdef float a = sin(dlat/2)**2 + cos(rlat1) * cos(rlat2) * sin(dlon/2)**2

    cdef float c = 2 * asin(sqrt(a))
    cdef float km = 6371 * c
    return km


In [33]:
%%time
for _ in range(10_000):
    cython_haversine(1,2,3,4)

CPU times: user 4.37 ms, sys: 56 µs, total: 4.43 ms
Wall time: 5.02 ms


In [34]:
%%time
for _ in range(10_000):
    haversine(1,2,3,4)

CPU times: user 26.5 ms, sys: 2.67 ms, total: 29.2 ms
Wall time: 31.6 ms


In [35]:
%%time
for _ in range(10_000):
    jit_haversine(1,2,3,4)

CPU times: user 10.2 ms, sys: 2.81 ms, total: 13 ms
Wall time: 20.3 ms


In [36]:
@vectorize([float64(float64, float64, float64, float64)],cache=True)
def numba_cython_haversine(lon1,lat1,lon2,lat2):
    return cython_haversine(lon1,lat1,lon2,lat2)

In [37]:
%%time
numba_cython_haversine(df["pickup_longitude"], df["pickup_latitude"], 
            df["dropoff_longitude"], df["dropoff_latitude"])

CPU times: user 173 ms, sys: 14 ms, total: 187 ms
Wall time: 253 ms


0         1.031069
1         8.449763
2         1.389644
3         2.799485
4         1.998886
            ...   
499995    1.404761
499996    0.994068
499997    7.858963
499998    0.994381
499999    1.040102
Length: 500000, dtype: float64